# 2b - Create Landsat 8 time series for McKinley Mine seeding units
This notebook creates a Landsat 8 time series of NDVI or other band ratios for each seeding unit created by notebook 1.

In [ ]:
# Import libraries
import os
import datetime as dt
from datetime import date

import numpy as np
import pandas as pd
import geopandas as gpd
import rioxarray as rxr
import rasterstats as rs
import matplotlib.pyplot as plt
import shutil
import configparser

# Import custom module for this project
from vegrestoretools import FisToDataframe, EvalScripts, FisClean, GeoseriesBbox

from sentinelhub import SHConfig, FisRequest, BBox, Geometry, CRS, WcsRequest, CustomUrlParam, \
    DataCollection, HistogramType, MimeType
from sentinelhub.time_utils import iso_to_datetime, parse_time

In [ ]:
# Read SentinelHub configuration ID's from your local ini file
config_ini = configparser.ConfigParser()
config_ini.read('landsat_setup.ini')

# Populate a SentinelHub configuration object with the ID's from the local ini file
config = SHConfig()
config.instance_id = config_ini['Config']['instance_id'] # Instance ID for the Configuration
config.sh_client_id = config_ini['Config']['sh_client_id'] # Credentials from the OAuth client
config.sh_client_secret = config_ini['Config']['sh_client_secret']
config.save()

In [ ]:
# Other parameters
startDate = '2020-01-01'

In [ ]:
# Set working directory
os.chdir(os.path.join('D:/',
                      'McKinley'))

# Directory to store downloaded Landsat 8 data
landsat_downloads_dir = os.path.join('landsat_downloads')

# Check if directory exists
if not os.path.exists(landsat_downloads_dir):
    os.makedirs(landsat_downloads_dir)

## Restore seeding unit geodataframe

In [ ]:
%store -r mckinley_seeding_subset_PARlo

# Add the index as a separate column
mckinley_seeding_subset_PARlo.reset_index(inplace=True)
mckinley_seeding_subset_PARlo = mckinley_seeding_subset_PARlo.rename(columns = {'index':'SU_ID'})

mckinley_seeding_subset_PARlo

## Query SentinelHub for available Landsat 8 data

In [ ]:
# Configure SentinelHub DataCollection; Landsat8 is a bit fussy
DataCollection.define_from(
    DataCollection.LANDSAT8,
    "LOTL1",
    service_url="https://services-uswest2.sentinel-hub.com",
    wfs_id="DSS12",  # https://www.sentinel-hub.com/develop/api/ogc/standard-parameters/wfs/
    api_id="LOTL1",
    catalog_id="landsat-8-l1c",
    processing_level="L1",
    bands=("B01", "B02", "B03", "B04", "B05", "B06", "B07", "B10"),
)

In [ ]:
# Convert seeding polygons to WGS84
seeding_aois = mckinley_seeding_subset_PARlo
seeding_aois = seeding_aois.to_crs("EPSG:4326")

# Save the seeding polygons to a shapefile for use in rasterstats
seeding_aois_path = os.path.join(landsat_downloads_dir, 'seeding_aois.shp')
seeding_aois.to_file(seeding_aois_path)

# Get bbox for seeding units
AOI_coords_ndarray = seeding_aois.total_bounds.round(decimals=2)
AOI_coords = list(AOI_coords_ndarray)
AOI_bbox = BBox(bbox=AOI_coords, crs=CRS.WGS84)

In [ ]:
# Build SentinelHub query to get all available Landsat 8 datasets for polygon
landsat8_dates_query = WcsRequest(layer='ALL_BANDS',  # Name of layer from service endpoint
                                  data_collection=DataCollection.LOTL1,
                                  bbox=AOI_bbox,
                                  time=(startDate, date.today()),
                                  maxcc=0.05,
                                  resx='20m', resy='20m',
                                  config=config)

# Get the available dates
landsat8_dates = landsat8_dates_query.get_dates()
landsat8_dates

## Download Landsat 8 data and perform zonal stats

In [ ]:
# Retrieve custom script to pass to SentinelHub from vegrestoretools module
evalscript = EvalScripts('Landsat_8_NDVI')

# Create list to store each date
output_list = []

# Loop over each unique date and download the data
for date in landsat8_dates:

    # Status message
    print('Currently processing: ' + str(date))

    # Try/except in case there are issues downloading specific datasets
    try: 
        # Build the download WCS request
        landsat8_wcs_download = WcsRequest(layer='ALL_BANDS',
                                           data_collection=DataCollection.LOTL1,
                                           bbox=AOI_bbox,
                                           time=date.date(),
                                           maxcc=0.05,
                                           resx='50m', resy='50m',
                                           data_folder=landsat_downloads_dir,
                                           image_format=MimeType.TIFF,
                                           config=config,
                                           custom_url_params={
                                               CustomUrlParam.EVALSCRIPT: evalscript},
                                           time_difference=dt.timedelta(seconds=1))

        # Run the download request
        landsat8_wcs_download.save_data()

        # Get download list
        downloads = landsat8_wcs_download.get_filename_list()

        # Open the dataset
        landsat8_data_path = landsat8_wcs_download.get_filename_list()
        landsat8_data = rxr.open_rasterio(os.path.join(
            'landsat_downloads', landsat8_data_path[0])).squeeze()

        # Run zonal stats
        landsat8_ndvi_stats_raw = rs.zonal_stats(seeding_aois_path,
                                                 landsat8_data.values,
                                                 affine=landsat8_data.rio.transform(),
                                                 geojson_out=True,  # Optional if you want to plot
                                                 copy_properties=True,  # Optional if you want to plot
                                                 nodata=0,  # Ignore values = 0 - Thought Question: Why are we doing that?
                                                 stats=["mean", "max"])

        # Convert to a GDF
        landsat8_ndvi_stats_df = gpd.GeoDataFrame.from_features(
            landsat8_ndvi_stats_raw)

        # Convert to DF
        landsat8_ndvi_stats_df = pd.DataFrame(landsat8_ndvi_stats_df.drop(columns='geometry'))

        # Rename columns
        landsat8_ndvi_stats_df.rename(columns={"max": "ndvi_max",
                                               "mean": "ndvi_mean"},
                                      inplace=True)

        # Add date field to df
        landsat8_ndvi_stats_df['landsat8_date'] = date.date()

        # Append the df to the temporary list
        output_list.append(landsat8_ndvi_stats_df)
    
    except:
        print('Error encountered downloading this scene')
        continue
    
# Concatenate all the results into a single DF
landsat8_ndvi_stats_all_df = pd.concat(output_list, axis=0)
landsat8_ndvi_stats_all_df

In [ ]:
# TESTING - delete downloads directory
#shutil.rmtree(landsat_downloads_dir)

## Plot the results

In [ ]:
# Setup plot
fig, ax = plt.subplots(figsize=(15, 10))

# Build list of unique sites for legend
sites = fis_df_all.ID.unique()

fis_df_all.reset_index().groupby('ID').plot(x='date', y='mean', ax=ax)

# Legend and labels
plt.legend(sites)

plt.xlabel("Date", fontsize=20)
plt.ylabel("NDVI", fontsize=20)

ax.set_title(
    "Sentinel 2 NDVI time series for McKinley Mine Vegetation Management Units", fontsize=20)

plt.setp(ax.get_xticklabels(), rotation=90, horizontalalignment='right')

plt.show()